# ADF parser API
## Imports to get the resource files

In [1]:
from vibrav.base import resource
import lzma
import os

Uncompress the resource file to read.

In [2]:
comp = resource('adf-ethane-ts-freq.t21.ascii.xz')
decomp = comp.split(os.sep)[-1][:-3]
with open(decomp, 'wb') as new_file, lzma.LZMAFile(comp, 'rb') as file:
    for data in iter(lambda : file.read(100 *1024), b''):
        new_file.write(data)

## Read the ASCII formatted ADF TAPE21 file

In [4]:
from vibrav.adf import Tape21

Read the atom and frequency tables.

In [5]:
ed = Tape21(decomp)

### Parse the frequency data frame

The `r_mass` column is calculated on the fly when parsing the mass-weighted normal modes with the formula,
$$ \mu_i = \left(\sum_k^{3N}l_{CARTk,i}^2\right)^{-1} = \left(\sum_k^{3N}\left(\frac{l_{MWC k,i}}{\sqrt{m_k}}\right)^2\right)^{-1} = N_i^2$$
The reason that we must use the ASCII formatted TAPE21 file from ADF rather than the output is that ADF prints the non-mass-weighted normalized cartesians. Looking at the equation shown above the $l_{CARTk,i}$ matrix elements are indeed non-mass-weighted cartesian normal modes. However, they are not normalized and the information of the reduced masses can still be extracted. Performing the same operation on the normal modes from the ADF frequency output will yield all of the reduced masses to be one.

To calculate the reduced masses the program will need the latest developers version of both the [exa](https://github.com/exa-analytics/exa) and [exatomic](https://github.com/exa-analytics/exatomic) packages.

In [6]:
ed.parse_frequency()
ed.frequency

,dx,dy,dz,frequency,freqdx,r_mass,symbol,label,ir_int,frame
frequency,,,,,,,,,,
0,8.326673e-16,-4.454770e-15,-0.013449,-239.036547,0,1.008159,C,0,0,0
1,-2.123302e-15,9.454243e-16,0.013449,-239.036547,0,1.008159,C,1,0,0
2,-6.875667e-02,4.020776e-01,-0.014577,-239.036547,0,1.008159,H,2,0,0
3,3.825878e-01,-1.414938e-01,-0.014577,-239.036547,0,1.008159,H,3,0,0
4,-3.138311e-01,-2.605838e-01,-0.014577,-239.036547,0,1.008159,H,4,0,0
...,...,...,...,...,...,...,...,...,...,...
139,-2.049873e-02,1.589500e-04,0.002457,3044.853587,17,1.100672,H,3,0,0
140,2.820427e-01,-3.476600e-01,-0.172553,3044.853587,17,1.100672,H,4,0,0
141,7.915730e-02,-2.686373e-01,0.107965,3044.853587,17,1.100672,H,5,0,0


### Parse the atom data frame

In [7]:
ed.parse_atom()
ed.atom

,symbol,set,label,x,y,z,Z,frame
atom,,,,,,,,
0,C,0,0,0.000000,0.000000,1.450715,6.0,0
1,C,1,1,0.000000,0.000000,-1.450715,6.0,0
2,H,2,2,1.905940,0.346368,2.224450,1.0,0
3,H,3,3,-0.653007,-1.823777,2.224450,1.0,0
4,H,4,4,-1.252933,1.477409,2.224450,1.0,0
5,H,5,5,-0.653007,1.823777,-2.224450,1.0,0
6,H,6,6,-1.252933,-1.477409,-2.224450,1.0,0
7,H,7,7,1.905940,-0.346368,-2.224450,1.0,0


Remove the uncompressed resource file

In [8]:
os.remove(decomp)